<img src="slides/8_30.png">

In [ ]:
import pandas as pd
import numpy as np
loan = pd.read_csv('data/loan_data.csv')

# 使用しない列を削除
df = loan.drop(['Unnamed: 0', 'status', 'loan_amnt', 'term', 'annual_inc', 'dti',
                  'revol_bal', 'revol_util', 'purpose', 'home_ownership',
                  'delinq_2yrs_zero', 'pub_rec_zero', 'open_acc', 'grade'], axis=1)

# ダミー変数化
df['outcome'] = df['outcome'].map(lambda x: 1 if x == 'default' else 0)
df = pd.get_dummies(data=df)

# 説明変数、目的変数を作成
y = 'outcome'
X = df.drop([y], axis=1)
Y = df[y]

### ホールドアウトにより予測精度を検証

```train_test_split()``` によりデータセットを分割

- ```test_size=0.3``` 30% をテストデータとして分割
- ```X_train, X_test``` 教師データ
- ```Y_train, Y_test``` テストデータ

```model.fit(X_train, Y_train)```

- 教師データで決定木を作成

```Y_predict = model.predict(X_test)```<br>
```Y_proba = model.predict_proba(X_test)```

- テストデータで目的変数を予測

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split

def classify(depth):
    # データを分割
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
    # 決定木を作成
    model = DecisionTreeClassifier(criterion='entropy', max_depth=depth)
    model.fit(X_train, Y_train)

    # 正確度、適合度、再現率（感度）、AUCの算出
    Y_predict = model.predict(X_test)
    Y_proba = model.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(Y_test, Y_proba[:, 1])
    print('depth={:2}: 正確度={:.3f}, 適合度={:.3f}, 再現率={:.3f}, AUC={:.3f}'.format(
        depth, accuracy_score(Y_test, Y_predict), precision_score(Y_test, Y_predict),
        recall_score(Y_test, Y_predict), auc(fpr, tpr)))

### 決定木の深さの数値を変更して実行

2 から 16 未満まで値を変更しながら、決定木を作成し、精度、AUCを確認

In [ ]:
for d in range(2, 16, 1):
    classify(d)

- max_depth=5~8 あたりでAUCは最大化
- max_depth=10 以上でAUCは低下

データセットの分割はランダムに行われるため常に同じ結果は出ない

<hr>

<img src="slides/8_31.png">